In [1]:
import tensorflow as tf
import os
import sys
import go_datafile_reader
import model
import numpy as np
import model_eval
reload(model)
reload(go_datafile_reader)
reload(model_eval)

<module 'model_eval' from 'model_eval.pyc'>

In [2]:
train_dir = "/home/justin/Programming/GoAI/MovePredictionCNN/data/input/train"
test_dir = "/home/justin/Programming/GoAI/MovePredictionCNN/data/input/test"
ckpt_path = "/home/justin/Programming/GoAI/MovePredictionCNN/data/working/board_eval_cnn_5layer.ckpt"

In [3]:
train_files = []
for subdir, dirs, files in os.walk(train_dir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".dat"):
            train_files.append(filepath)

test_files = []
for subdir, dirs, files in os.walk(test_dir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".dat"):
            test_files.append(filepath)

print "num train: %d, num test: %d" %(len(train_files), len(test_files))

num train: 14724, num test: 149


In [4]:
#note you may have to change the os limit for number of open files to use the RandomAccessFileReader
#you can do this with the command "sudo ulimit -n 20000"
#if sudo can't find the ulimit command try the following below
#sudo sh -c "ulimit -n 20000 && exec su $LOGNAME"

#reader = go_datafile_reader.RandomAccessFileReader(train_files)
test_reader = go_datafile_reader.GoDatafileReader(test_files)

test_reader.num_epochs = 0
test_features = []
test_targets = []
test_move_numbers = []
while(test_reader.num_epochs == 0):
    test_move_numbers.append(test_reader.move_index)
    final_state, _, feature_cube = test_reader.read_sample()
    test_features.append(feature_cube)
    test_targets.append(final_state)

In [5]:
x, ownership = model.place_holders()
y_conv = model.model(x)
loss = model.loss_function(ownership, y_conv)
train_op = model.train_step(loss)

prediction = tf.round(y_conv)
correct_prediction = tf.equal(ownership, prediction)
correct_count = tf.reduce_sum(tf.cast(correct_prediction, "float"))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver(tf.all_variables())
saver.restore(sess, ckpt_path)

In [6]:
import matplotlib.pyplot as plt
import numpy as np

In [9]:
accuracies = []
y_trues = []
y_preds = []
feature_cubes = []
for i in xrange(len(test_features)):
    if i%500 == 0:
        print i
    x_, y_ = test_features[i], test_targets[i]
    y_val, y_pred, acc = sess.run([y_conv, prediction, accuracy], feed_dict={x:[x_], ownership:[y_]})
    accuracies.append(acc)
    y_trues.append(np.reshape(y_, [19,19]))
    y_preds.append(np.reshape(y_pred[0], [19,19]))
    feature_cubes.append(x_)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500


In [10]:
idx = 3500
y_pred = y_preds[idx]
y_true = y_trues[idx]
feature_cube = feature_cubes[idx]

print test_move_numbers[idx], accuracies[idx]
model_eval.print_info(feature_cube, y_pred, y_true)

    

212 0.961219
*0*1*0****0*1*1****   1111000000011100000   1111100000011110000
*1*110****010**010*   1111100000010000000   1111100000010100000
*11100***0*10000100   1111000000010000000   1111000000010000000
11000******1*11100*   1100000000111111000   1100000000110111000
10*******001****110   1000000000011111110   1000000000011111110
*0******0*101*1*10*   0000000000001111111   1000000000001111111
***0******1011**100   0000000000001111111   0000000000001111111
*01*1****1001*10*11   0000000000001111111   0000000000001111111
0***000*100101*1***   0000000000000111111   0000000000000111111
10000101101*001****   1000010000000011111   1000010000000011111
11101110001*0111***   1110111000000111111   1110111000000111111
*0111**001*0**0011*   1111111000000000111   1111111000000100111
*****1001*1*00*001*   1111110000000000011   1111110000000000011
******10010**10*001   1111111000000000001   1111111000000000001
******1*0100001001*   1111111000000010011   1111111000000010011
**1**01*1010*01011*   11111

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame()
df["Move Num"] = test_move_numbers
df["Accuracy"] = accuracies
grouped = df.groupby("Move Num").aggregate(np.mean)


In [13]:
grouped.plot()
plt.title("Accuracy by move number, 5layer 64filters")
plt.xlabel("Move Number")
plt.ylabel("Accuracy")
plt.show()

In [14]:
plot = plt.hist(df[df["Move Num"] == 250]["Accuracy"].values, bins = 40)
plt.show()